# Download Strava Ride Activity

This code will download a single ride from Strava and save it to a structured (pipe|carat) text format. Strava (the company) is fairly sensitive to mass downloads of data, so this code is intended to download rides one at a time. This code assumes that you have already downloaded a master list of rides by athlete, so you already have the Strava activity ID for the ride that you want to download.

In [7]:
#custom library to import Strava ride data
#https://github.com/hozn/stravalib
#http://pythonhosted.org/stravalib/
from stravalib import Client

#other libraries
import csv
from datetime import datetime
from datetime import timedelta

In [8]:
athleteid = 750228          #My Strava athlete ID
activityid = 839792480      #Strava-generated activity ID
ridedate = '2017-01-21'     #date of activity (used for filename only)

In [9]:
#Strava tracks distances in meters and speed in meters/second
#let's be backwards and convert m/s to mph. for reference, 1 mph ~= 2.24 m/s
def convert_ms_to_mph(speed_val):
    return format(float(speed_val * 2.23693629), '.1f')

In [10]:
#create txt file
#uses pipe | carat format to reduce conflicts with characters in source data
ofile = open('./Data/' + str(athleteid) + '_' + str(activityid) + '_' + ridedate + '.txt', 'w')
writer = csv.writer(ofile, delimiter='|', quotechar='^', quoting=csv.QUOTE_ALL)

In [11]:
#write header row for txt file
header = 'AthleteID', 'ActivityID', 'StartTime', 'TotalElapsedTime', 'TotalDistanceMeters', 'MaxSpeedMPH', \
                'MeasuredTime', 'Latitude', 'Longitude', 'AltitudeMeters', 'DistanceMeters', 'CurrentSpeedMPH', \
                'CurrentGrade'
writer.writerow(header);

In [12]:
#this is my super-secret Strava developer access token
client = Client(access_token=[REDACTED])        #no need to publish this to the whole world

In [13]:
%%capture --no-stdout
#connect to Strava and request data on specified activity ID
#Strava will occasionally throw up (non-fatal) warning messages. no need to be concerned about them 
strava_ride = client.get_activity(activityid)
max_speed = convert_ms_to_mph(strava_ride.max_speed)           #max speed is stored at top level of Strava activity

In [14]:
#streams are raw Strava data. think of them as columnar data that are similarly indexed
#so that they can be stitched back together
stream_types = ['time','distance','latlng','altitude','grade_smooth','velocity_smooth']
streams = client.get_activity_streams(activityid, types=stream_types)

stream_time = streams["time"].data 
stream_distance = streams["distance"].data 
stream_latlng = streams["latlng"].data 
stream_altitude = streams["altitude"].data 
stream_grade = streams["grade_smooth"].data 
stream_velocity = streams["velocity_smooth"].data

stream_tuple = zip(stream_time, stream_distance, stream_latlng, stream_altitude, stream_grade, stream_velocity)

In [15]:
for (time,distance,latlng,altitude,grade,velocity) in stream_tuple:
    #Strava tracks incremental ride time as seconds from start time (instead of as an absolute time value)
    current_time = strava_ride.start_date_local + timedelta(seconds=time)
    current_speed = convert_ms_to_mph(velocity)
    
    activity_tuple = athleteid, activityid, strava_ride.start_date_local, strava_ride.elapsed_time, \
        float(strava_ride.distance), max_speed, current_time, latlng[0], latlng[1], \
        altitude, distance, current_speed, grade
    writer.writerow(activity_tuple)
    
ofile.close()
print('All done!')

All done!
